# Animaciones e Interactividad

Esta clase continúa el tema de transformaciones y se centra en dos ideas clave:
1. **Animaciones** con `imageio`.
2. **Interactividad** con `ipywidgets` (sliders para parámetros).


In [1]:
from animaciones import *

## 1. Animaciones (frames + GIF con `imageio`)

Tres ejemplos básicos con un triángulo 2D:
1. Traslación uniforme.  
2. Rotación alrededor del centroide.  
3. Traslación y luego rotación (el orden importa).

### 1.1 Generadores de frames para el triángulo

Cada generador:
- Prepara una carpeta para los frames (limpia PNGs previos).
- Dibuja y guarda cada frame con `plt.savefig(...)`.
- Configura los ejes con aspecto igual y límites fijos para evitar "saltos".

In [2]:
# -----------------------------------------------------------------------------
# EJEMPLOS DE USO 
# -----------------------------------------------------------------------------

# 1) Triángulo trasladado
generar_frames_traslacion(carpeta_frames="frames_traslacion", n_frames=60)
crear_gif_desde_frames(
    carpeta_frames="frames_traslacion",
    salida_gif="triangulo_traslacion.gif",
    duracion_frame=0.03  # ~33 fps
)

GIF creado en: triangulo_traslacion.gif (frames: 60, duración/frame: 0.03s)


In [3]:
# 2) Triángulo rotando sobre su centroide
generar_frames_rotacion(carpeta_frames="frames_rotacion", n_frames=60)
crear_gif_desde_frames(
    carpeta_frames="frames_rotacion",
    salida_gif="triangulo_rotacion.gif",
    duracion_frame=0.03
)

GIF creado en: triangulo_rotacion.gif (frames: 60, duración/frame: 0.03s)


In [4]:
# 3) Triángulo trasladado y luego rotado
generar_frames_traslado_luego_rotacion(carpeta_frames="frames_mix", n_frames=60)
crear_gif_desde_frames(
    carpeta_frames="frames_mix",
    salida_gif="triangulo_traslado_luego_rotacion.gif",
    duracion_frame=0.03
)

GIF creado en: triangulo_traslado_luego_rotacion.gif (frames: 60, duración/frame: 0.03s)


## 2. Widgets simples 

### 2.1 Elipse interactiva variando $a$ y $b$

La elipse canónica satisface $\tfrac{x^2}{a^2} + \tfrac{y^2}{b^2} = 1$.

In [5]:
# Interactividad con widgets (deslizadores)
from ipywidgets import interact, interactive, FloatSlider, IntSlider, VBox, HBox, Layout

In [10]:
def trazar_elipse(a: float = 2.0, b: float = 1.0) -> None:
    """
    Dibuja una elipse con semiejes 'a' (en x) y 'b' (en y).

    Parámetros
    ----------
    a : float
        Semieje horizontal (>0).
    b : float
        Semieje vertical (>0).
    Regresa
    -------
    None
        Solo realiza la gráfica.
    """
    if a <= 0 or b <= 0:
        # Validación básica de parámetros
        raise ValueError("Se requiere a > 0 y b > 0 para una elipse válida.")
    
    n = 400 # Número de muestras angulares para interpolar la curva.

    # Parametrización estándar de la elipse con ángulo t ∈ [0, 2π]
    t = np.linspace(0, 2*np.pi, n)          # np.linspace crea una malla uniforme de n puntos
    x = a * np.cos(t)                        # Eje x escalado por 'a'
    y = b * np.sin(t)                        # Eje y escalado por 'b'

    # Creamos una figura nueva por cada llamada para evitar sobreescrituras
    plt.figure(figsize=(5, 5))               # figsize en pulgadas
    plt.plot(x, y, lw=2)                     # lw controla el grosor de línea

    # gca() obtiene el eje actual; set_aspect('equal') iguala escalas en x e y.
    # adjustable='box' le dice a Matplotlib que ajuste el cuadro para mantener esa proporción.
    plt.gca().set_aspect("equal", adjustable="box")

    # Establecemos límites simétricos basados en el mayor semieje para encuadre limpio
    m = max(a, b) * 1.1                      # margen del 10% para que no quede justo en el borde
    plt.xlim(-m, m)                          # set_xlim fija límites en eje x
    plt.ylim(-m, m)                          # set_ylim fija límites en eje y

    plt.title(f"Elipse con a={a:.2f}, b={b:.2f}")  # Título
    plt.grid(True)                           # grid muestra cuadrícula de referencia
    plt.show()                               # Renderiza la figura en el notebook


def elipse_interactiva():
    """
    Crea sliders para manipular a y b y observar la elipse en tiempo real.
    """
    # interact construye una UI mínima con sliders a partir de las anotaciones del callable
    return interact(
        trazar_elipse,
        a=FloatSlider(value=2.0, min=0.2, max=5.0, step=0.1, description='a'),
        b=FloatSlider(value=1.0, min=0.2, max=5.0, step=0.1, description='b')
    )

# Ejecutar:
elipse_interactiva()

interactive(children=(FloatSlider(value=2.0, description='a', max=5.0, min=0.2), FloatSlider(value=1.0, descri…

<function __main__.trazar_elipse(a: float = 2.0, b: float = 1.0) -> None>

### 2.2 Paraboloide interactivo: $z = c_x x^2 + c_y y^2$

Se añaden **dos** coeficientes independientes $c_x$ y $c_y$ para controlar la curvatura en cada dirección.


In [12]:
def trazar_paraboloide(cx: float = 1.0, cy: float = 1.0, rango: float = 2.0, n: int = 50) -> None:
    """
    Dibuja la superficie z = c_x x^2 + c_y y^2 con malla regular.

    Parámetros
    ----------
    cx : float
        Coeficiente cuadrático en x.
    cy : float
        Coeficiente cuadrático en y.
    rango : float
        Rango simétrico de visualización en x, y ([-rango, rango]).
    n : int
        Resolución de la malla (más grande = más puntos).

    Regresa
    -------
    None
        Solo grafica.
    """
    # Construimos mallas 2D para X e Y con np.meshgrid
    xs = np.linspace(-rango, rango, n)       # Valores de x
    ys = np.linspace(-rango, rango, n)       # Valores de y
    X, Y = np.meshgrid(xs, ys)               # Malla cartesiana (X, Y)

    Z = cx * (X**2) + cy * (Y**2)            # Evaluamos z = c_x x^2 + c_y y^2

    # Creamos figura y eje 3D
    fig = plt.figure(figsize=(6, 5))         # Figura de 6x5 pulgadas
    ax = fig.add_subplot(111, projection='3d') # Eje 3D con proyección '3d'
    ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True, alpha=0.9)  # Superficie suave

    ax.set_title(f"Paraboloide: z = {cx:.2f} x^2 + {cy:.2f} y^2")  # Título con parámetros
    ax.set_xlabel("x")                          # Etiqueta eje x
    ax.set_ylabel("y")                          # Etiqueta eje y
    ax.set_zlabel("z")                          # Etiqueta eje z
    plt.show()                                    # Mostrar figura


def paraboloide_interactivo():
    """
    Crea sliders para cx, cy, el rango y la resolución n.
    """
    return interact(
        trazar_paraboloide,
        cx=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description='c_x'),
        cy=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description='c_y'),
        rango=FloatSlider(value=2.0, min=1.0, max=5.0, step=0.5, description='rango'),
        n=IntSlider(value=50, min=20, max=120, step=5, description='resol.')
    )

# Ejecutar:
paraboloide_interactivo()

interactive(children=(FloatSlider(value=1.0, description='c_x', max=2.0, min=-2.0), FloatSlider(value=1.0, des…

<function __main__.trazar_paraboloide(cx: float = 1.0, cy: float = 1.0, rango: float = 2.0, n: int = 50) -> None>

### 2.3 Toroide interactivo $(a, b, c)$

Parametrización con sección elíptica:
$$
\begin{aligned}
x(u, v) &= (a + b\cos v)\cos u,\\
y(u, v) &= (a + b\cos v)\sin u,\\
z(u, v) &= c\sin v,
\end{aligned}
\quad u, v \in [0, 2\pi].
$$


In [13]:
def trazar_toroide(a: float = 2.0, b: float = 0.7, c: float = 0.7,
                    nu: int = 60, nv: int = 40) -> None:
    """
    Dibuja un toroide con parámetros (a, b, c) y resoluciones (nu, nv).

    Parámetros
    ----------
    a : float
        Radio mayor (distancia desde el centro del tubo al eje).
    b : float
        Escala en cos(v) para la sección transversal.
    c : float
        Escala en sin(v) para la sección transversal (si b=c, sección circular).
    nu : int
        Puntos en la dirección u (revolución).
    nv : int
        Puntos en la dirección v (sección).

    Regresa
    -------
    None
        Solo grafica.
    """
    if a <= 0:
        raise ValueError("Se requiere a > 0 para un toroide válido.")

    # Mallado de los parámetros u, v
    u = np.linspace(0, 2*np.pi, nu)
    v = np.linspace(0, 2*np.pi, nv)
    U, V = np.meshgrid(u, v)

    # Ecuaciones paramétricas del toroide
    X = (a + b*np.cos(V)) * np.cos(U)
    Y = (a + b*np.cos(V)) * np.sin(U)
    Z = c * np.sin(V)

    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True, alpha=0.9)

    ax.set_title(f"Toroide con a={a:.2f}, b={b:.2f}, c={c:.2f}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    # Ajuste simple de límites para encuadre completo del toroide
    R = a + max(abs(b), abs(c)) + 0.5
    ax.set_xlim(-R, R)
    ax.set_ylim(-R, R)
    ax.set_zlim(-R, R)
    plt.show()


def toroide_interactivo():
    """
    Sliders para (a, b, c) y resoluciones de malla (nu, nv).
    """
    return interact(
        trazar_toroide,
        a=FloatSlider(value=2.0, min=0.5, max=4.0, step=0.1, description='a'),
        b=FloatSlider(value=0.7, min=0.2, max=2.0, step=0.1, description='b'),
        c=FloatSlider(value=0.7, min=0.2, max=2.0, step=0.1, description='c'),
        nu=IntSlider(value=60, min=20, max=120, step=10, description='nu'),
        nv=IntSlider(value=40, min=20, max=120, step=10, description='nv')
    )

# Ejecutar:
toroide_interactivo()

interactive(children=(FloatSlider(value=2.0, description='a', max=4.0, min=0.5), FloatSlider(value=0.7, descri…

<function __main__.trazar_toroide(a: float = 2.0, b: float = 0.7, c: float = 0.7, nu: int = 60, nv: int = 40) -> None>

### 2.4 Cíclide interactivo $(a, b, c, d)$

Parametrización tipo cíclide (superficie de Dupin):

$$
\begin{aligned}
x(u, v) &= \frac{d\,(c - a\cos u \cos v) + b^2 \cos u}{a - c\cos u \cos v}, \\
y(u, v) &= \frac{b \sin u \,(a - d\cos v)}{a - c\cos u \cos v}, \\
z(u, v) &= \frac{b \sin v \,(c \cos u - d)}{a - c\cos u \cos v},
\end{aligned}
\quad u, v \in [0, 2\pi].
$$

In [ ]:
def superficie_cicloide(a: float, b: float, c: float, d: float,
                       nu: int = 60, nv: int = 40,
                       tol: float = 1e-6):
    """
    Genera las mallas (X, Y, Z) de la superficie tipo toro/cíclide:

        x = ( d*(c - a*cos(u)*cos(v)) + b^2*cos(u) ) / ( a - c*cos(u)*cos(v) )
        y =   b*sin(u)*(a - d*cos(v))              / ( a - c*cos(u)*cos(v) )
        z =   b*sin(v)*(c*cos(u) - d)              / ( a - c*cos(u)*cos(v) )

    con u ∈ [0, 2π], v ∈ [0, 2π].

    Parámetros
    ----------
    a, b, c, d : float
        Parámetros geométricos de la superficie.
    nu, nv : int
        Resoluciones de la malla en u y v, respectivamente.
    tol : float
        Tolerancia mínima para el |denominador|; valores menores se recortan para
        evitar división por cero.

    Regresa
    -------
    (X, Y, Z) : np.ndarray, np.ndarray, np.ndarray
        Mallas 2D listas para graficar con plot_surface.
    """
    # Mallado de parámetros
    u = np.linspace(0.0, 2.0*np.pi, nu)
    v = np.linspace(0.0, 2.0*np.pi, nv)
    U, V = np.meshgrid(u, v)

    cu, su = np.cos(U), np.sin(U)
    cv, sv = np.cos(V), np.sin(V)

    # Denominador con protección numérica
    denom = a - c * cu * cv
    # Evitar división por cero; recortar donde el valor absoluto sea muy pequeño
    denom = np.where(np.abs(denom) < tol, np.sign(denom) * tol, denom)

    # Fórmulas paramétricas
    X = ( d*(c - a*cu*cv) + (b**2)*cu ) / denom
    Y = ( b*su*(a - d*cv) ) / denom
    Z = ( b*sv*(c*cu - d) ) / denom

    return X, Y, Z


def trazar_ciclide(a: float = 2.0, b: float = 0.7, c: float = 0.7, d: float = 1.2,
                           nu: int = 80, nv: int = 60, tol: float = 1e-6) -> None:
    """
    Dibuja la superficie tipo cíclide/toroide con los parámetros dados.

    Parámetros
    ----------
    a, b, c, d : float
        Parámetros de la superficie.
    nu, nv : int
        Resoluciones de malla para u y v.
    tol : float
        Tolerancia para el denominador.

    Regresa
    -------
    None
        Solo grafica.
    """
    X, Y, Z = superficie_cicloide(a, b, c, d, nu=nu, nv=nv, tol=tol)

    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111, projection='3d')

    # Superficie
    ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True, alpha=0.95)

    # Etiquetas y título
    ax.set_title(f"Cíclide/Toroide: a={a:.2f}, b={b:.2f}, c={c:.2f}, d={d:.2f}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    # Límites automáticos con margen
    max_r = 1.05 * np.max(np.abs([X, Y, Z]))
    ax.set_xlim(-max_r, max_r)
    ax.set_ylim(-max_r, max_r)
    ax.set_zlim(-max_r, max_r)

    plt.show()


def ciclide_interactiva():
    """
    Interfaz interactiva con sliders para (a, b, c, d) y resolución (nu, nv).
    """
    return interact(
        trazar_ciclide,
        a=FloatSlider(value=2.0, min=0.5, max=4.0, step=0.1, description='a'),
        b=FloatSlider(value=0.7, min=0.2, max=2.0, step=0.05, description='b'),
        c=FloatSlider(value=0.7, min=0.2, max=2.0, step=0.05, description='c'),
        d=FloatSlider(value=1.2, min=0.2, max=3.0, step=0.05, description='d'),
        nu=IntSlider(value=80, min=30, max=200, step=10, description='nu'),
        nv=IntSlider(value=60, min=30, max=200, step=10, description='nv'),
        tol=FloatSlider(value=1e-6, min=1e-8, max=1e-3, step=1e-8, readout_format=".0e", description='tol')
    )

# Ejecutar:
ciclide_interactiva()


interactive(children=(FloatSlider(value=2.0, description='a', max=4.0, min=0.5), FloatSlider(value=0.7, descri…

<function __main__.trazar_ciclide(a: float = 2.0, b: float = 0.7, c: float = 0.7, d: float = 1.2, nu: int = 80, nv: int = 60, tol: float = 1e-06) -> None>

## 3 Widgets de sistemas físicos

### 3.1 Péndulo 

Modelamos el ángulo del péndulo con la **aproximación de ángulo pequeño**:
$$ \theta(t) = A \cos(\omega t) + B \sin(\omega t), \quad \omega = \sqrt{\tfrac{g}{L}} $$

Partimos de una configuración inicial: $\theta(0) = \theta_0$ y $\dot\theta(0) = \omega_0$.

In [14]:
def angulo_pendulo(t: float, theta0: float, w0: float, L: float, g: float = 9.81) -> float:
    """
    Ángulo del péndulo simple bajo aproximación de ángulo pequeño.

    Parámetros
    ----------
    t : float
        Tiempo (s).
    theta0 : float
        Ángulo inicial (rad).
    w0 : float
        Velocidad angular inicial (rad/s).
    L : float
        Longitud de la cuerda (m).
    g : float
        Aceleración de la gravedad (m/s^2).

    Regresa
    -------
    float
        Ángulo θ(t) en radianes.
    """
    omega = math.sqrt(g / L)     # Frecuencia natural del péndulo
    A = theta0                   # Constante A por condición inicial θ(0)=θ0
    B = w0 / omega               # Constante B por condición inicial θ'(0)=w0
    return A * math.cos(omega * t) + B * math.sin(omega * t)


def pendulo_estatico_en_t(L: float = 1.5, theta0: float = 0.6, w0: float = 0.0,
                          g: float = 9.81, t: float = 0.0) -> None:
    """
    Dibuja el péndulo en el instante de tiempo 't' usando la solución θ(t).

    Parámetros
    ----------
    L : float
        Longitud de la cuerda (m).
    theta0 : float
        Ángulo inicial (rad).
    w0 : float
        Velocidad angular inicial (rad/s).
    g : float
        Aceleración de la gravedad (m/s^2).
    t : float
        Tiempo (s) en el cual evaluar θ(t).

    Regresa
    -------
    None
        Solo grafica el estado en el tiempo dado.
    """
    # Calculamos el ángulo en el tiempo t
    theta = angulo_pendulo(t, theta0=theta0, w0=w0, L=L, g=g)

    # Posición del bob (extremo de la cuerda) en coordenadas cartesianas
    x = L * math.sin(theta)   # Desplazamiento horizontal: L*sin(θ)
    y = -L * math.cos(theta)  # Desplazamiento vertical:  -L*cos(θ) (hacia abajo)

    # Construcción de la figura
    fig, ax = plt.subplots(figsize=(5, 5))   # Nueva figura y eje
    ax.plot([0, x], [0, y], lw=2)            # Segmento desde el pivote (0,0) hasta el bob (x,y)
    bob = plt.Circle((x, y), radius=0.08*L, fill=True, alpha=0.7)  # Circunferencia para la masa
    ax.add_patch(bob)                        # add_patch añade el círculo al eje
    ax.plot([0], [0], marker='o')            # Marca el pivote como un punto

    # Ajustes de encuadre
    ax.set_xlim(-L*1.2, L*1.2)               # set_xlim define límites horizontales
    ax.set_ylim(-L*1.2, L*0.2)               # set_ylim define límites verticales

    # Igualar escala de unidades en x e y para que el péndulo no se deforme
    plt.gca().set_aspect("equal", adjustable="box")  # Ver comentario en sección de la elipse

    ax.set_title(f"Péndulo en t={t:.2f} s (θ={theta:.2f} rad)")  # Título con instante y ángulo
    plt.grid(True)                           # Mostrar cuadrícula
    plt.show()                               # Renderizar la figura


def pendulo_interactivo():
    """
    Interfaz interactiva para controlar L, θ0, w0, g y t (tiempo).
    """
    return interact(
        pendulo_estatico_en_t,
        L=FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description='L (m)'),
        theta0=FloatSlider(value=0.6, min=-1.2, max=1.2, step=0.05, description='θ0 (rad)'),
        w0=FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.1, description='ω0 (rad/s)'),
        g=FloatSlider(value=9.81, min=1.0, max=20.0, step=0.1, description='g (m/s²)'),
        t=FloatSlider(value=0.0, min=0.0, max=10.0, step=0.05, description='t (s)')
    )

# Ejecutar:
pendulo_interactivo()


interactive(children=(FloatSlider(value=1.5, description='L (m)', max=3.0, min=0.5), FloatSlider(value=0.6, de…

<function __main__.pendulo_estatico_en_t(L: float = 1.5, theta0: float = 0.6, w0: float = 0.0, g: float = 9.81, t: float = 0.0) -> None>

### 3.2 Widgets de trayectoria de un proyectil 
Ecuaciones (sin resistencia del aire, altura inicial $y_0$):
$$
\begin{aligned}
x(t) &= v_0 \cos(\theta)\, t, \\
y(t) &= y_0 + v_0 \sin(\theta)\, t - \tfrac{1}{2} g t^2.
\end{aligned}
$$

Tiempo de vuelo (primera raíz positiva de $y(t)=0$):
$$
t_{\text{vuelo}} = \frac{v_0 \sin\theta + \sqrt{(v_0 \sin\theta)^2 + 2 g y_0}}{g}.
$$

Alcance horizontal:
$$
R = v_0 \cos\theta\; t_{\text{vuelo}}.
$$


In [15]:
def tiempo_de_vuelo(v0: float, theta_rad: float, y0: float = 0.0, g: float = 9.81) -> float:
    """
    Calcula el tiempo total de vuelo (t > 0) para lanzamiento con altura y0.

    Parámetros
    ----------
    v0 : float
        Velocidad inicial (m/s).
    theta_rad : float
        Ángulo inicial (rad).
    y0 : float
        Altura inicial (m).
    g : float
        Aceleración de la gravedad (m/s^2).

    Regresa
    -------
    float
        Tiempo de vuelo. Si no hay impacto con y=0, puede regresar NaN.
    """
    vy0 = v0 * math.sin(theta_rad)           # Componente vertical inicial
    disc = vy0**2 + 2*g*y0                   # Discriminante de la cuadrática en y(t)
    if disc < 0:
        return float('nan')
    return (vy0 + math.sqrt(disc)) / g       # Raíz positiva


def trayectoria_proyectil(v0: float, theta_grados: float, y0: float = 0.0, g: float = 9.81, n: int = 300):
    """
    Genera (x, y) desde t=0 hasta t_vuelo.

    Regresa
    -------
    (x: np.ndarray, y: np.ndarray, t_vuelo: float, alcance: float)
    """
    theta = math.radians(theta_grados)       # Conversión grados→radianes
    t_vuelo = tiempo_de_vuelo(v0, theta, y0, g)
    # if not math.isfinite(t_vuelo) or t_vuelo <= 0:
    #     # Caso patológico: devolvemos arrays mínimos para evitar errores
    #     return np.array([0.0]), np.array([y0]), float('nan'), float('nan')
    t = np.linspace(0, t_vuelo, n)           # Malla de tiempo uniforme
    x = v0 * math.cos(theta) * t             # Posición x(t)
    y = y0 + v0 * math.sin(theta) * t - 0.5 * g * t**2  # Posición y(t)
    return x, y, t_vuelo, x[-1]              # alcance = x(t_vuelo)


def proyectil_interactivo_con_tiempo():
    """
    Interfaz interactiva: v0, θ, y0, g y tiempo t (para marcar posición instantánea).
    """

    def graficar(v0: float = 20.0, theta: float = 45.0, y0: float = 0.0, g: float = 9.81, t: float = 1.0):
        # Calculamos la trayectoria completa y magnitudes clave
        x, y, t_v, R = trayectoria_proyectil(v0, theta, y0, g, n=400)

        # Definimos figura y eje
        fig, ax = plt.subplots(figsize=(7, 4.5))   # Tamaño algo mayor para texto fuera del eje
        ax.plot(x, y, lw=2)                        # Curva de la trayectoria
        ax.axhline(0, ls='--', alpha=0.5)          # Línea de "suelo" en y=0

        # Marcamos la posición a tiempo t (limitando t al rango [0, t_v])
        t_eff = min(max(t, 0.0), t_v)              # t "efectivo" acotado
        x_t = v0 * math.cos(math.radians(theta)) * t_eff
        y_t = y0 + v0 * math.sin(math.radians(theta)) * t_eff - 0.5 * g * t_eff**2
        ax.plot([x_t], [y_t], marker='o')          # Punto actual del proyectil

        # Igualar escala de ejes para no deformar la parábola
        ax.set_aspect('equal', adjustable='box')   # Misma razón de unidades en x e y
        ax.set_xlabel("x (m)")                    # Etiqueta eje x
        ax.set_ylabel("y (m)")                    # Etiqueta eje y
        ax.set_title(f"Proyectil: v0={v0:.1f} m/s, θ={theta:.1f}°, y0={y0:.1f} m")  # Título

        # Ajustar límites para que las leyendas no se traslapen con la curva
        # Añadimos margen superior del 20% sobre el máximo de y
        y_max = max(float(np.max(y)), y0) if len(y) > 0 else y0
        ax.set_ylim(bottom=min(-1.0, float(np.min(y)) - 0.1*abs(y_max)), top=y_max * 1.2)

        # Colocar texto (t_vuelo y alcance) FUERA del área del eje: usamos fig.text
        # fig.text(x_frac, y_frac, texto, ...) coloca texto en coordenadas de la figura (0..1)
        info = (f"t_vuelo ≈ {t_v:.2f} s\n"
                f"Alcance ≈ {R:.2f} m\n"
                f"t = {t_eff:.2f} s, (x(t), y(t)) ≈ ({x_t:.2f}, {y_t:.2f})")
        fig.text(0.02, 0.98, info, va='top')       # Texto en la esquina superior izquierda de la figura

        plt.grid(True)                             # Cuadrícula de referencia
        plt.show()                                 # Mostrar

    return interact(
        graficar,
        v0=FloatSlider(value=20.0, min=1.0, max=60.0, step=1.0, description='v0 (m/s)'),
        theta=FloatSlider(value=45.0, min=0.0, max=89.0, step=1.0, description='θ (°)'),
        y0=FloatSlider(value=0.0, min=0.0, max=10.0, step=0.5, description='y0 (m)'),
        g=FloatSlider(value=9.81, min=1.0, max=20.0, step=0.1, description='g (m/s²)'),
        t=FloatSlider(value=1.0, min=0.0, max=10.0, step=0.05, description='t (s)')
    )

# Ejecutar:
proyectil_interactivo_con_tiempo()


interactive(children=(FloatSlider(value=20.0, description='v0 (m/s)', max=60.0, min=1.0, step=1.0), FloatSlide…

<function __main__.proyectil_interactivo_con_tiempo.<locals>.graficar(v0: float = 20.0, theta: float = 45.0, y0: float = 0.0, g: float = 9.81, t: float = 1.0)>

## 4. Ejercicios

1. **Orden de transformaciones (frames + GIF)**: Repite el ejemplo 1.3 pero ahora **rota primero** alrededor del **centroide** y **luego traslada**. Genera los frames y crea el GIF final. Compara visualmente ambos GIFs y explica la diferencia.
2. **Centro de rotación**: Modifica el generador de frames de rotación para rotar alrededor de un **vértice** del triángulo. ¿Cómo cambia la trayectoria de los vértices?
3. **Efecto de los límites**: Cambia los límites del eje en los generadores de frames (1.1–1.3) y explica por qué mantenerlos fijos evita “saltos” en la animación.
4. **Péndulo amortiguado (widget)**: Agrega un factor de decaimiento $e^{-\alpha t}$ a $\theta(t)$ y explora el efecto con un nuevo slider $\alpha$.
5. **Elipse**:¿Qué ocurre con la parametrización si $a=b$? ¿Qué ocurre con la parametrización si $a$ o $b \to 0^+$?
6. **Paraboloide**: Explora $c_x$ y $c_y$ negativos. ¿Cómo interpretarías geométricamente la superficie resultante?
7. **Óptimo del alcance (proyectil)**: Con $y_0=0$ y $g$ constante, usa el widget para **estimar** el ángulo que maximiza $R$. 
